In [1]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import itertools
from eval_utils import evaluate

In [ ]:
import pandas as pd
import os
data_file = os.path.join(os.getcwd(), '../Data/processedfile.csv')
data = pd.read_csv(data_file)

In [ ]:
# Split the email into individual words
words = data['decoded']
unique_words = list(itertools.chain.from_iterable(words.values))

In [ ]:
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting words into numbers.
le.fit(unique_words)
X = words.apply(le.transform)
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
del X, Y

In [30]:
spam_proba = len(y_train[y_train == 1])/len(y_train)
#Create a Gaussian Classifier
model = GaussianNB(priors=[spam_proba, 1 - spam_proba])
# Train the model using the training sets
model.fit(X_train,y_train)

In [13]:
pred = model.predict(X_test, y_test)

In [ ]:
evaluate(pred, y_test)